### Imports

In [1]:
# !pip install scapy
from scapy.all import *
import pandas as pd
import numpy as np
import binascii
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

### Load Data into Dataframe

In [2]:
clean_all_packets = pd.read_csv('clean_all_packets.csv')

In [3]:
clean_all_packets.head()

,Unnamed: 0,index,version,ihl,tos,len,id,flags,frag,ttl,...,51.15.37.44_y,52.1.134.178_y,52.70.139.21_y,52.84.124.115_y,52.84.126.158_y,52.84.126.62_y,77.222.63.27_y,8.8.8.8_y,83.217.75.51_y,92.38.135.151_y
0,0,0,4,5,0,196,19331,NaN,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,4,5,0,196,19331,NaN,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,2,4,5,0,62,341,NaN,0,128,...,0,0,0,0,0,0,0,0,0,0
3,3,3,4,5,0,62,341,NaN,0,128,...,0,0,0,0,0,0,0,0,0,0
4,4,4,4,5,0,202,14015,NaN,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
list(clean_all_packets.columns)[0:30]

['Unnamed: 0',
 'index',
 'version',
 'ihl',
 'tos',
 'len',
 'id',
 'flags',
 'frag',
 'ttl',
 'proto',
 'chksum',
 'src',
 'dst',
 'options',
 'time',
 'payload',
 'payload_raw',
 'payload_hex',
 'target',
 'S',
 'R',
 'A',
 'FA',
 'SA',
 'RA',
 'PA',
 'FPA',
 'SAE',
 'SEC']

### Set X and y values

In [7]:
# Sets x vales
X = clean_all_packets.drop(['Unnamed: 0','index','version','ihl','tos','time','id','flags','frag','ttl',
                             'proto','chksum','src','dst','options','payload','payload_raw','payload_hex','target'], axis=1)
# sets y value
y = clean_all_packets['target']

#### Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, test_size=.3, random_state=42)

#### Scaling

In [9]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

/Users/dorrisb/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/dorrisb/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/dorrisb/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """


#### Baseline

In [10]:
# examine the class distribution of the training set
y_train.value_counts(normalize=True)

1    0.50261
0    0.49739
Name: target, dtype: float64

In [11]:
# calculate baseline (infected pcap)
y_train.mean()

0.5026100960737491

In [12]:
# calculate percentage of zeros (clean pcap)
1 - y_train.mean()

0.49738990392625093

## Model

### Linear Regression

In [16]:
from sklearn.linear_model import LinearRegression

# instantiate Linear Regression model
linreg = LinearRegression()

# fir model with training data
linreg.fit(X_train, y_train)

# print training score
print('Train Score:', linreg.score(X_train, y_train))

# print testing score
print('Test Score:', linreg.score(X_test, y_test))

Train Score: 0.2434351001395718
Test Score: -2901135039192.707


### Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

# instantiating Logistic Regression Model
logreg = LogisticRegression()

# fit the model on training data
logreg.fit(X_train_sc, y_train)

# evaluates model on the training data
print('Train Score:', logreg.score(X_train_sc, y_train))

# evaluates model on the testing data
print('Test Score :', logreg.score(X_test_sc, y_test))

# create predicted y values
y_pred_logreg = logreg.predict(X_test_sc)

/Users/dorrisb/anaconda3/envs/dsi/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train Score: 0.7397678680513133
Test Score : 0.7408487204405572


<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)>

In [29]:
logreg.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)>

#### Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_logreg)

array([[6807, 1005],
       [2995, 4628]])

In [39]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_logreg).ravel()
print('False Negative:', fn,'--', 'False Positive:',fp)
print('')
print('True Negative:',tn,' --', 'True Positive:', tp) 

False Negative: 2995 -- False Positive: 1005

True Negative: 6807  -- True Positive: 4628


In [ ]:
# from sklearn.svm import SVC

# # finding best tranformation on the columns 

# # instantiate SVC model
# svc = SVC()

# # fit model training data (X scaled)
# svc.fit(X_train_sc, y_train)

# # evaluates model on the training data
# print('Train Score:', svc.score(X_train_sc, y_train))

# # evaluates model on the testing data
# print('Test Score :', svc.score(X_test_sc, y_test))

# # create predicted y values
# y_pred_svc = svc.predict(X_test_sc)